<a href="https://colab.research.google.com/github/JaeKwonLim/Yeonsu_decline/blob/main/save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from geoband import API
import geopandas as gpd
import pandas as pd
import datetime
import numpy as np

In [ ]:
# csv to local
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [ ]:
# 파일 불러오기

data_03 = pd.read_excel('3_연수구_거주인구_격자매핑.xlsx', sheet_name = None)

data_05 = pd.read_excel('5_연수구_성연령별_주민등록인구.xlsx', sheet_name = None)

data_08 = pd.read_excel('8_연수구_고충민원_격자매핑.xlsx', sheet_name = None)

data_09 = pd.read_excel('9_연수구_민원현황_격자매핑.xlsx', sheet_name = None)
data_10 = pd.read_excel('10_연수구_건축물대장_표제부.xlsx', sheet_name = None)


data_13 = pd.read_csv('13_연수구_차량등록현황_격자매핑.csv')
data_14 = pd.read_csv('14_연수구_112신고수_격자매핑.csv')


data_17 = pd.read_excel('17_연수구_단독다가구_매매_실거래가.xlsx', sheet_name = None)



data_21 = pd.read_excel('21_연수구_단독다가구_전월세_실거래가.xlsx', sheet_name = None)

data_23 = pd.read_excel('23_연수구_상업업무용_매매_실거래가.xlsx', sheet_name = None)



data_27 = pd.read_excel('27_연수구_교통사고_격자매핑.xlsx', sheet_name = None)
data_28 = gpd.read_file('28_연수구_격자_100X100.geojson', sheet_name = None)

In [ ]:
# 전입전출
data_01 = pd.read_excel('1_연수구_전입현황.xlsx', sheet_name = None)
data_02 = pd.read_excel('2_연수구_전출현황.xlsx', sheet_name = None)

concatted_01 = pd.concat(data_01, ignore_index=True)
concatted_01.columns = ['emd_nm', 'sd_nm', 'sgg_nm', 'base_date', 'come_count', 'come_family', 'come_pop']
index_01 = concatted_01[concatted_01['sd_nm'] == '관내'].index
concatted_01 = concatted_01.drop(index_01)
concatted_01['where'] = concatted_01['sd_nm'] + " " + concatted_01['sgg_nm']
concatted_01 = concatted_01.astype({'base_date' : 'str'})
concatted_01['date'] = concatted_01['base_date'].str[:4]
concatted_01 = concatted_01.astype({'date' : 'int'})
concatted_01.drop(['sd_nm', 'sgg_nm', 'base_date'], axis=1, inplace=True)

concatted_02 = pd.concat(data_02, ignore_index=True)
concatted_02.columns = ['emd_nm', 'sd_nm', 'sgg_nm', 'base_date', 'out_count', 'out_family', 'out_pop']
index_02 = concatted_02[concatted_02['sd_nm'] == '관내'].index
concatted_02 = concatted_02.drop(index_02)
concatted_02['where'] = concatted_02['sd_nm'] + " " + concatted_02['sgg_nm']
concatted_02 = concatted_02.astype({'base_date' : 'str'})
concatted_02['date'] = concatted_02['base_date'].str[:4]
concatted_02 = concatted_02.astype({'date' : 'int'})
concatted_02.drop(['sd_nm', 'sgg_nm', 'base_date'], axis=1, inplace=True)

concatted_03 = pd.concat([concatted_01, concatted_02], ignore_index = True)

def func(x):
    a = {}
    a['count'] = x['come_count'].sum() - x['out_count'].sum()
    a['family'] = x['come_family'].sum() - x['out_family'].sum()
    a['pop'] = x['come_pop'].sum() - x['out_pop'].sum()
    

    return pd.Series(a, index=['count', 'family', 'pop'])

df = concatted_03.groupby(['date', 'emd_nm', 'where']).apply(func).reset_index()

In [ ]:
# 취약계층(독거노인, 수급자) 현황

data_04 = pd.read_csv('4_연수구_독거노인.csv',)
data_07 = pd.read_excel('7_연수구_기초생활수급대상자.xlsx', sheet_name = None)

data_04 = data_04.astype({'base_date' : 'str'})
data_04['date'] = data_04['base_date'].str[:4]
data_04 = data_04.astype({'date' : 'int'})
data_04.drop(['base_date'], axis=1, inplace=True)
data_04.columns = ['emd_nm', 'loneliness', 'M_loneliness', 'W_loneliness', 'date']

concatted_07 = pd.concat(data_07)
concatted_07.reset_index(inplace = True)
concatted_07['date'] = concatted_07['level_0'].str[:4]
concatted_07.drop(['level_0', 'level_1'], axis=1, inplace=True)
index_07 = concatted_07[concatted_07['emd_nm'] == '합 계'].index
concatted_07 = concatted_07.drop(index_07)
concatted_07.columns = ['emd_nm', 'basic_pop', 'date']
concatted_07 = concatted_07.astype({'date' : 'int'})

concatted_04 = pd.concat([data_04, concatted_07], ignore_index = True)

def func(x):
    b = {}
    b['loneliness'] = x['loneliness'].sum()
    b['M_loneliness'] = x['M_loneliness'].sum()
    b['W_loneliness'] = x['W_loneliness'].sum()
    b['basic_pop'] = x['basic_pop'].sum()
    return pd.Series(b, index=['loneliness', 'M_loneliness', 'W_loneliness', 'basic_pop'])

df = concatted_04.groupby(['date', 'emd_nm']).apply(func).reset_index()

In [ ]:
# 매매 동향 분석

data_15 = pd.read_excel('15_연수구_아파트_매매_실거래가.xlsx', sheet_name = None)
data_16 = pd.read_excel('16_연수구_연립다세대_매매_실거래가.xlsx', sheet_name = None)
data_18 = pd.read_excel('18_연수구_오피스텔_매매_실거래가.xlsx', sheet_name = None)

concatted_15 = pd.concat(data_15)
concatted_15.reset_index(inplace = True)
concatted_16 = pd.concat(data_16)
concatted_16.reset_index(inplace = True)
concatted_18 = pd.concat(data_18)
concatted_18.reset_index(inplace = True)

# -----------------------------------------------------------------------------------

name_split = concatted_15['bjd_nm'].str.split(" ")

concatted_15 = concatted_15.astype({'cntrc_ym' : 'str'})
concatted_15['contrc_date'] = concatted_15['cntrc_ym'].str[:4]
concatted_15 = concatted_15.astype({'contrc_date' : 'int'})
concatted_15['emd_nm'] = name_split.str.get(2)
concatted_15.drop(['level_0','level_1','bunji', 'road_nm', 'cmplx_nm', 'cntrc_day', 'floor', 'cntrc_ym', 'bjd_nm'], axis=1, inplace=True)

# 60㎡ 이하는 소형, 60㎡ ~ 85㎡ 이하는 중소형, 85㎡ ~ 135㎡ 이하는 중대형, 135㎡ 초과는 대형 - 국토해양부
condition = [
    (concatted_15['exclusive_area'] > 135),
    (concatted_15['exclusive_area'] >= 85),
    (concatted_15['exclusive_area'] > 60),
    (concatted_15['exclusive_area'] <= 60)]
choicelist = ['L', 'ML', 'MS', 'S']
concatted_15['size'] = np.select(condition, choicelist, default='Not Specified')

# 이상치 제거
index_15a = concatted_15[concatted_15['trade_price'] == '00,000'].index
index_15b = concatted_15[concatted_15['trade_price'] == '00,500'].index
concatted_15 = concatted_15.drop(index_15a)
concatted_15 = concatted_15.drop(index_15b)
concatted_15 = concatted_15.astype({'trade_price' : 'int'})


def func(x):
    b = {}
    b['exclusive_area'] = x['exclusive_area'].mean()
    b['trade_price'] = x['trade_price'].mean()
    b['cnstr_year'] = x['cnstr_year'].mean()
    return pd.Series(b, index=['exclusive_area', 'trade_price', 'cnstr_year'])

df_15 = concatted_15.groupby(['contrc_date', 'emd_nm', 'size']).apply(func).reset_index()

# -----------------------------------------------------------------------------------

name_split = concatted_16['bjd_nm'].str.split(" ")

concatted_16 = concatted_16.astype({'cntrc_ym' : 'str'})
concatted_16['contrc_date'] = concatted_16['cntrc_ym'].str[:4]
concatted_16 = concatted_16.astype({'contrc_date' : 'int'})
concatted_16['emd_nm'] = name_split.str.get(2)
concatted_16.drop(['level_0','level_1','bunji', 'road_nm', 'bld_nm', 'land_area', 'cntrc_day', 'floor', 'cntrc_ym', 'bjd_nm'], axis=1, inplace=True)

# 60㎡ 이하는 소형, 60㎡ ~ 85㎡ 이하는 중소형, 85㎡ ~ 135㎡ 이하는 중대형, 135㎡ 초과는 대형 - 국토해양부
condition = [
    (concatted_16['exclusive_area'] > 135),
    (concatted_16['exclusive_area'] >= 85),
    (concatted_16['exclusive_area'] > 60),
    (concatted_16['exclusive_area'] <= 60)]
choicelist = ['L', 'ML', 'MS', 'S']
concatted_16['size'] = np.select(condition, choicelist, default='Not Specified')

concatted_16 = concatted_16.astype({'trade_price' : 'int'})


def func(x):
    b = {}
    b['exclusive_area'] = x['exclusive_area'].mean()
    b['trade_price'] = x['trade_price'].mean()
    b['cnstr_year'] = x['cnstr_year'].mean()
    return pd.Series(b, index=['exclusive_area', 'trade_price', 'cnstr_year'])

df_16 = concatted_16.groupby(['contrc_date', 'emd_nm', 'size']).apply(func).reset_index()

# -----------------------------------------------------------------------------------

name_split = concatted_18['bjd_nm'].str.split(" ")

concatted_18 = concatted_18.astype({'cntrc_ym' : 'str'})
concatted_18['contrc_date'] = concatted_18['cntrc_ym'].str[:4]
concatted_18 = concatted_18.astype({'contrc_date' : 'int'})
concatted_18['emd_nm'] = name_split.str.get(2)
concatted_18.drop(['level_0','level_1','bunji', 'road_nm', 'cmplx_nm', 'cntrc_day', 'floor', 'cntrc_ym', 'bjd_nm'], axis=1, inplace=True)

# 60㎡ 이하는 소형, 60㎡ ~ 85㎡ 이하는 중소형, 85㎡ ~ 135㎡ 이하는 중대형, 135㎡ 초과는 대형 - 국토해양부
condition = [
    (concatted_18['exclusive_area'] > 135),
    (concatted_18['exclusive_area'] >= 85),
    (concatted_18['exclusive_area'] > 60),
    (concatted_18['exclusive_area'] <= 60)]
choicelist = ['L', 'ML', 'MS', 'S']
concatted_18['size'] = np.select(condition, choicelist, default='Not Specified')

concatted_18 = concatted_18.astype({'trade_price' : 'int'})


def func(x):
    b = {}
    b['exclusive_area'] = x['exclusive_area'].mean()
    b['trade_price'] = x['trade_price'].mean()
    b['cnstr_year'] = x['cnstr_year'].mean()
    return pd.Series(b, index=['exclusive_area', 'trade_price', 'cnstr_year'])

df_18 = concatted_18.groupby(['contrc_date', 'emd_nm', 'size']).apply(func).reset_index()

In [ ]:
# 전월세 동향 분석

data_19 = pd.read_excel('19_연수구_아파트_전월세_실거래가.xlsx', sheet_name = None)
data_20 = pd.read_excel('20_연수구_연립다세대_전월세_실거래가.xlsx', sheet_name = None)
data_21 = pd.read_excel('21_연수구_단독다가구_전월세_실거래가.xlsx', sheet_name = None)
data_22 = pd.read_excel('22_연수구_오피스텔_전월세_실거래가.xlsx', sheet_name = None)

# -----------------------------------------------------------------------------------

concatted_19 = pd.concat(data_19)
concatted_19.reset_index(inplace = True)
concatted_20 = pd.concat(data_20)
concatted_20.reset_index(inplace = True)
concatted_22 = pd.concat(data_22)
concatted_22.reset_index(inplace = True)

# -----------------------------------------------------------------------------------

name_split = concatted_19['bjd_nm'].str.split(" ")

concatted_19 = concatted_19.astype({'cntrc_ym' : 'str'})
concatted_19['contrc_date'] = concatted_19['cntrc_ym'].str[:4]
concatted_19 = concatted_19.astype({'contrc_date' : 'int'})
concatted_19['emd_nm'] = name_split.str.get(2)
concatted_19.drop(['level_0','level_1','bunji', 'road_nm', 'cmplx_nm', 'cntrc_day', 'floor', 'cntrc_ym', 'bjd_nm'], axis=1, inplace=True)

# 60㎡ 이하는 소형, 60㎡ ~ 85㎡ 이하는 중소형, 85㎡ ~ 135㎡ 이하는 중대형, 135㎡ 초과는 대형 - 국토해양부
condition = [
    (concatted_19['exclusive_area'] > 135),
    (concatted_19['exclusive_area'] >= 85),
    (concatted_19['exclusive_area'] > 60),
    (concatted_19['exclusive_area'] <= 60)]
choicelist = ['L', 'ML', 'MS', 'S']
concatted_19['size'] = np.select(condition, choicelist, default='Not Specified')

def func(x):
    b = {}
    b['deposit'] = x['deposit'].mean()
    b['m_rent'] = x['m_rent'].mean()
    b['cnstr_year'] = x['cnstr_year'].mean()
    b['exclusive_area'] = x['exclusive_area'].mean()
    return pd.Series(b, index=['exclusive_area', 'deposit', 'm_rent', 'cnstr_year'])

df_19 = concatted_19.groupby(['contrc_date', 'emd_nm','rent_type', 'size']).apply(func).reset_index()

# -----------------------------------------------------------------------------------

name_split = concatted_20['bjd_nm'].str.split(" ")

concatted_20 = concatted_20.astype({'cntrc_ym' : 'str'})
concatted_20['contrc_date'] = concatted_20['cntrc_ym'].str[:4]
concatted_20 = concatted_20.astype({'contrc_date' : 'int'})
concatted_20['emd_nm'] = name_split.str.get(2)
concatted_20.drop(['level_0','level_1','bunji', 'road_nm', 'bld_nm', 'cntrc_day', 'floor', 'cntrc_ym'], axis=1, inplace=True)

# 60㎡ 이하는 소형, 60㎡ ~ 85㎡ 이하는 중소형, 85㎡ ~ 135㎡ 이하는 중대형, 135㎡ 초과는 대형 - 국토해양부
condition = [
    (concatted_20['exclusive_area'] > 135),
    (concatted_20['exclusive_area'] >= 85),
    (concatted_20['exclusive_area'] > 60),
    (concatted_20['exclusive_area'] <= 60)]
choicelist = ['L', 'ML', 'MS', 'S']
concatted_20['size'] = np.select(condition, choicelist, default='Not Specified')

def func(x):
    b = {}
    b['deposit'] = x['deposit'].mean()
    b['m_rent'] = x['m_rent'].mean()
    b['cnstr_year'] = x['cnstr_year'].mean()
    b['exclusive_area'] = x['exclusive_area'].mean()
    return pd.Series(b, index=['exclusive_area', 'deposit', 'm_rent', 'cnstr_year'])

df_20 = concatted_20.groupby(['contrc_date', 'emd_nm','rent_type', 'size']).apply(func).reset_index()

# -----------------------------------------------------------------------------------

name_split = concatted_22['bjd_nm'].str.split(" ")

concatted_22 = concatted_22.astype({'cntrc_ym' : 'str'})
concatted_22['contrc_date'] = concatted_22['cntrc_ym'].str[:4]
concatted_22 = concatted_22.astype({'contrc_date' : 'int'})
concatted_22['emd_nm'] = name_split.str.get(2)
concatted_22.drop(['level_0','level_1','bunji', 'road_nm', 'cmplx_nm', 'cntrc_day', 'floor', 'cntrc_ym'], axis=1, inplace=True)

# 60㎡ 이하는 소형, 60㎡ ~ 85㎡ 이하는 중소형, 85㎡ ~ 135㎡ 이하는 중대형, 135㎡ 초과는 대형 - 국토해양부
condition = [
    (concatted_22['exclusive_area'] > 135),
    (concatted_22['exclusive_area'] >= 85),
    (concatted_22['exclusive_area'] > 60),
    (concatted_22['exclusive_area'] <= 60)]
choicelist = ['L', 'ML', 'MS', 'S']
concatted_22['size'] = np.select(condition, choicelist, default='Not Specified')

def func(x):
    b = {}
    b['deposit'] = x['deposit'].mean()
    b['m_rent'] = x['m_rent'].mean()
    b['cnstr_year'] = x['cnstr_year'].mean()
    b['exclusive_area'] = x['exclusive_area'].mean()
    return pd.Series(b, index=['exclusive_area', 'deposit', 'm_rent', 'cnstr_year'])

df_22 = concatted_22.groupby(['contrc_date', 'emd_nm','rent_type', 'size']).apply(func).reset_index()

In [ ]:
#기타 가격

# data_23 = pd.read_excel('23_연수구_상업업무용_매매_실거래가.xlsx', sheet_name = None)
data_24 = pd.read_excel('24_연수구_토지_실거래가.xlsx', sheet_name = None)
data_25 = pd.read_excel('25_연수구_분양입주권_실거래가.xlsx', sheet_name = None)
data_26 = pd.read_excel('26_연수구_개별공시지가.xlsx', sheet_name = None)

concatted_24 = pd.concat(data_24)
concatted_24.reset_index(inplace = True)
concatted_25 = pd.concat(data_25)
concatted_25.reset_index(inplace = True)
concatted_26 = pd.concat(data_26)
concatted_26.reset_index(inplace = True)

# -----------------------------------------------------------------------------------

index_24 = concatted_24[concatted_24['use_area'] == '보전녹지지역'].index
concatted_24 = concatted_24.drop(index_24)
index_24 = concatted_24[concatted_24['use_area'] == '개발제한구역'].index
concatted_24 = concatted_24.drop(index_24)
index_24 = concatted_24[concatted_24['use_area'] == '자연녹지지역'].index
concatted_24 = concatted_24.drop(index_24)
index_24 = concatted_24[concatted_24['road_cndtn'] == '-'].index
concatted_24 = concatted_24.drop(index_24)


name_split = concatted_24['bjd_nm'].str.split(" ")

concatted_24 = concatted_24.astype({'cntrc_ym' : 'str'})
concatted_24['contrc_date'] = concatted_24['cntrc_ym'].str[:4]
concatted_24 = concatted_24.astype({'contrc_date' : 'int'})
concatted_24['emd_nm'] = name_split.str.get(2)
concatted_24.drop(['level_0','level_1','bjd_nm', 'bunji', 'road_cndtn', 'cntrc_ym', 'cntrc_day', 'share_type'], axis=1, inplace=True)

def func(x):
    b = {}
    b['cntrc_area'] = x['cntrc_area'].mean()
    b['trade_price'] = x['trade_price'].mean()
    return pd.Series(b, index=['cntrc_area', 'trade_price'])

df_24 = concatted_24.groupby(['contrc_date', 'emd_nm','use_area']).apply(func).reset_index()

# -----------------------------------------------------------------------------------

name_split = concatted_25['bjd_nm'].str.split(" ")

concatted_25 = concatted_25.astype({'cntrc_ym' : 'str'})
concatted_25['contrc_date'] = concatted_25['cntrc_ym'].str[:4]
concatted_25 = concatted_25.astype({'contrc_date' : 'int'})
concatted_25['emd_nm'] = name_split.str.get(2)
concatted_25.drop(['level_0','level_1','cmplx_nm', 'bunji', 'cntrc_ym', 'cntrc_day', 'floor'], axis=1, inplace=True)

def func(x):
    b = {}
    b['exclusive_area'] = x['exclusive_area'].mean()
    b['trade_price'] = x['trade_price'].mean()
    return pd.Series(b, index=['exclusive_area', 'trade_price'])

df_25 = concatted_25.groupby(['contrc_date', 'emd_nm','right_type']).apply(func).reset_index()

# -----------------------------------------------------------------------------------

name_split = concatted_26['bjd_nm'].str.split(" ")

index_26 = concatted_26[concatted_26['s_land_nm'] == '산'].index
concatted_26 = concatted_26.drop(index_26)

concatted_26['emd_nm'] = name_split.str.get(2)
concatted_26.drop(['level_0','level_1','bjd_cd','bjd_nm', 'unique_no', 'unique_no', 's_land_cd', 'jibun', 'month','office_date'], axis=1, inplace=True)

def func(x):
    b = {}
    b['land_value'] = x['land_value'].mean()
    return pd.Series(b, index=['land_value'])

df_26 = concatted_26.groupby(['year', 'emd_nm']).apply(func).reset_index()

In [ ]:
# 상권

data_11 = pd.read_csv('11_연수구_상권정보.csv')

data_11['business_type'].dropna(inplace = True)
data_11.dropna(subset = ['close_date'], inplace = True)

name_split = data_11['location_address'].str.split(" ")

data_11 = data_11.astype({'license_date' : 'str'})
data_11['license_date'] = data_11['license_date'].str[:4]
data_11 = data_11.astype({'license_date' : 'int'})
data_11 = data_11.astype({'close_date' : 'str'})
data_11['close_date'] = data_11['close_date'].str[:4]
data_11 = data_11.astype({'close_date' : 'int'})

data_11['emd_nm'] = name_split.str.get(2)
data_11.drop(['business_type','open_type', 'road_address','location_address', 'store_nm'], axis=1, inplace=True)

# -----------------------------------------------------------------------------------



In [ ]:
data_12 = pd.read_excel('12_연수구_학교현황.xlsx', sheet_name = None)
data_12 = pd.concat(data_12)
data_12.reset_index(inplace = True)